In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

In [14]:
base_athlete_url = "https://www.olympedia.org/athletes"
athlete_url = f"{base_athlete_url}/1"
response = requests.get(athlete_url, timeout=60)

soup = BeautifulSoup(response.content, "html.parser")

In [16]:
soup.find("table", {"class": "table"})

<table class="table">
<thead>
<tr>
<th>Games</th>
<th>Discipline (Sport) / Event</th>
<th>NOC / Team</th>
<th>Pos</th>
<th>Medal</th>
<th>As</th>
<th></th>
</tr>
</thead>
<tbody>
<tr class="active">
<td>
<a href="/editions/6">1912 Summer Olympics</a>
</td>
<td><a href="/sports/TEN">Tennis</a></td>
<td><a href="/countries/FRA"><img src="https://olympedia-flags.s3.eu-central-1.amazonaws.com/FRA.png" style="padding-right: 2px; vertical-align: middle"/>FRA</a></td>
<td></td>
<td></td>
<td>Jean-François Blanchy</td>
<td>
</td>
</tr>
<tr>
<td></td>
<td>
<a href="/results/44337">Singles, Men</a>
<small>(Olympic)</small>
</td>
<td></td>
<td>=17</td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td>
<a href="/results/44412">Doubles, Men</a>
<small>(Olympic)</small>
</td>
<td><a href="/athletes/1805052">Jean Montariol</a></td>
<td><span class="label label-primary"><abbrev title="Did Not Start">DNS</abbrev></span></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr class="active">
<td>
<a href="/e

In [39]:
table = soup.find("table", {"class": "table"})

df = pd.read_html(StringIO(str(table)))[0]

df['athlete_id'] = 1
df['NOC'] = None
df['Discipline'] = None

df

,Games,Discipline (Sport) / Event,NOC / Team,Pos,Medal,As,Unnamed: 6,athlete_id,NOC,Discipline
0,1912 Summer Olympics,Tennis,FRA,NaN,NaN,Jean-François Blanchy,NaN,1,None,None
1,NaN,"Singles, Men (Olympic)",NaN,=17,NaN,NaN,NaN,1,None,None
2,NaN,"Doubles, Men (Olympic)",Jean Montariol,DNS,NaN,NaN,NaN,1,None,None
3,1920 Summer Olympics,Tennis,FRA,NaN,NaN,Jean-François Blanchy,NaN,1,None,None
4,NaN,"Singles, Men (Olympic)",NaN,=32,NaN,NaN,NaN,1,None,None
5,NaN,"Doubles, Mixed (Olympic)",Jeanne Vaussard,=8,NaN,NaN,NaN,1,None,None
6,NaN,"Doubles, Men (Olympic)",Jacques Brugnon,4,NaN,NaN,NaN,1,None,None


In [40]:
rows_to_keep = df.index[df['Games'].isna()].tolist()
rows_with_noc = df.index[~df['Games'].isna()].tolist()

In [41]:
rows_to_keep

[1, 2, 4, 5, 6]

In [22]:
rows_with_noc

[0, 3]

In [42]:
df.loc[rows_with_noc, 'NOC'] = df.loc[rows_with_noc, 'NOC / Team']
df.loc[rows_with_noc, 'Discipline'] = df.loc[rows_with_noc, 'Discipline (Sport) / Event']

In [43]:
df

,Games,Discipline (Sport) / Event,NOC / Team,Pos,Medal,As,Unnamed: 6,athlete_id,NOC,Discipline
0,1912 Summer Olympics,Tennis,FRA,NaN,NaN,Jean-François Blanchy,NaN,1,FRA,Tennis
1,NaN,"Singles, Men (Olympic)",NaN,=17,NaN,NaN,NaN,1,None,None
2,NaN,"Doubles, Men (Olympic)",Jean Montariol,DNS,NaN,NaN,NaN,1,None,None
3,1920 Summer Olympics,Tennis,FRA,NaN,NaN,Jean-François Blanchy,NaN,1,FRA,Tennis
4,NaN,"Singles, Men (Olympic)",NaN,=32,NaN,NaN,NaN,1,None,None
5,NaN,"Doubles, Mixed (Olympic)",Jeanne Vaussard,=8,NaN,NaN,NaN,1,None,None
6,NaN,"Doubles, Men (Olympic)",Jacques Brugnon,4,NaN,NaN,NaN,1,None,None


In [44]:
columns_to_ffill = ['Games', 'NOC', 'As', 'Discipline']
df[columns_to_ffill] = df[columns_to_ffill].ffill()

df

,Games,Discipline (Sport) / Event,NOC / Team,Pos,Medal,As,Unnamed: 6,athlete_id,NOC,Discipline
0,1912 Summer Olympics,Tennis,FRA,NaN,NaN,Jean-François Blanchy,NaN,1,FRA,Tennis
1,1912 Summer Olympics,"Singles, Men (Olympic)",NaN,=17,NaN,Jean-François Blanchy,NaN,1,FRA,Tennis
2,1912 Summer Olympics,"Doubles, Men (Olympic)",Jean Montariol,DNS,NaN,Jean-François Blanchy,NaN,1,FRA,Tennis
3,1920 Summer Olympics,Tennis,FRA,NaN,NaN,Jean-François Blanchy,NaN,1,FRA,Tennis
4,1920 Summer Olympics,"Singles, Men (Olympic)",NaN,=32,NaN,Jean-François Blanchy,NaN,1,FRA,Tennis
5,1920 Summer Olympics,"Doubles, Mixed (Olympic)",Jeanne Vaussard,=8,NaN,Jean-François Blanchy,NaN,1,FRA,Tennis
6,1920 Summer Olympics,"Doubles, Men (Olympic)",Jacques Brugnon,4,NaN,Jean-François Blanchy,NaN,1,FRA,Tennis


In [45]:




# Rename 'Discipline (Sport) / Event' to 'Event' & 'NOC / Team' to 'Team' for clarity
df.rename(columns={'Discipline (Sport) / Event': 'Event', 'NOC / Team': 'Team'}, inplace=True)

# Drop the 'Unnamed: 6' column as it's not needed
df.drop(columns=['Unnamed: 6'], inplace=True, errors='ignore')

# columns = ['Games', 'NOC / Team', 'Pos', 'Medal', 'As', 'Discipline (Sport)', 'Event']
df.iloc[rows_to_keep]

,Games,Event,Team,Pos,Medal,As,athlete_id,NOC,Discipline
1,1912 Summer Olympics,"Singles, Men (Olympic)",NaN,=17,NaN,Jean-François Blanchy,1,FRA,Tennis
2,1912 Summer Olympics,"Doubles, Men (Olympic)",Jean Montariol,DNS,NaN,Jean-François Blanchy,1,FRA,Tennis
4,1920 Summer Olympics,"Singles, Men (Olympic)",NaN,=32,NaN,Jean-François Blanchy,1,FRA,Tennis
5,1920 Summer Olympics,"Doubles, Mixed (Olympic)",Jeanne Vaussard,=8,NaN,Jean-François Blanchy,1,FRA,Tennis
6,1920 Summer Olympics,"Doubles, Men (Olympic)",Jacques Brugnon,4,NaN,Jean-François Blanchy,1,FRA,Tennis


In [31]:
df.iloc[[1, 2]]

,Games,Event,Team,Pos,Medal,As,athlete_id,NOC,Discipline
1,1912 Summer Olympics,"Singles, Men (Olympic)",NaN,=17,NaN,Jean-François Blanchy,1,FRA,Tennis
2,1912 Summer Olympics,"Doubles, Men (Olympic)",Jean Montariol,DNS,NaN,Jean-François Blanchy,1,FRA,Tennis
